In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction
In this notebook, we will demonstrate how Google Sheets can be used as a simple medium for managing, updating, and evaluating Intents and Training Phrases in Dialogflow CX.

Specifically, we will show how to `update` **_Existing Intents_** and Training Phrases in Dialogflow CX using Google Sheets as a Source

## Prerequisites
- Ensure you have a GCP Service Account key with the Dialogflow API Admin privileges assigned to it

In [ ]:
#If you haven't already, make sure you install the `dfcx-scrapi` library

!pip install dfcx-scrapi

# Imports

In [ ]:
import pandas as pd
from dfcx_scrapi.tools.copy_util import CopyUtil
from dfcx_scrapi.tools.dataframe_functions import DataframeFunctions

# User Inputs
In the next section, we will collect runtime variables needed to execute this notebook.   
This should be the only cell of the notebook you need to edit in order for this notebook to run.

For this notebook, we'll need the following inputs:

- `creds_path`: Your local path to your GCP Service Account Credentials
- `agent_id`: Your Dialogflow CX Agent ID in String format
- `google_sheet_name`: The name of your Google Sheet
- `google_sheet_tab_read`:  The name of the tab in your Google Sheet to read the data from

In [ ]:
creds_path = '<YOUR_CREDS_PATH_HERE>'
agent_id = '<YOUR_AGENT_ID_HERE>'
flow = '<YOUR_FLOW_DISPLAY_NAME>'
google_sheet_name = 'My Google Sheet Name'
google_sheet_tab_write = 'Write To My Tab Name'

# CX to Sheets - Filtered by Intents in Scope of a Flow
Here, we will demonstrate how to extract all of the Intents and Training Phrases associated with a specific `Flow` inside of a Dialogflow CX Agent.

In our previous notebook example, we extracted _ALL_ of the Intents and Training Phrases associated with the Agent.   
But in some cases, you may only be interested in Intents that are _currently in use_ with `Flow A` or `Flow B`.   
The following code allows you to easily extract that information and move it to a Google Sheet for review.

## Prerequisites
- In order for the `DataframeFunctions` class to interact with Google Sheets, you *must* share your Google Sheet with your Service Account email address.

In [ ]:
cu = CopyUtil(creds_path=creds_path, agent_id=agent_id)
dffx = DataframeFunctions(creds_path)

flow_map = cu.flows.get_flows_map(reverse=True)
pages = cu.pages.list_pages(flow_map[flow])
resources = cu.get_page_dependencies(pages)

for key in resources.keys():
    if key == 'intents':
        intent_list = list(resources[key])
        
all_intents = cu.intents.list_intents()

final_intents = []
for intent in all_intents:
    if intent.name in intent_list:
        final_intents.append(intent)

df = pd.DataFrame()

for intent in final_intents:
    df = df.append(cu.intents.intent_proto_to_dataframe(intent))

# Push DataFrame to Google Sheets
dffx.dataframe_to_sheets(google_sheet_name, google_sheet_tab_write, df)

print('Total # of Intents = {}'.format(df.intent.nunique()))
print('Total # of Training Phrases = {}'.format(df.tp.nunique()))

# Final Thoughts and Wrap-Up
You should see your Google Sheet is now updated with the Intents and Training Phrases from your Dialogflow CX Agent that are in scope of the `Flow` that you specified.

If you want to create _additional_ filters before pushing the data to Google Sheets, you can manipulate the `df` variable to do things like: 
- Exclude 1 or more Intents
- Push Intents that contain > X # of Training Phrases to Tab A
- Push Intents that contain < Y # of Training Phrases to Tab B